In [ ]:
import requests
import datetime
import pandas as pd 
import numpy as np
#from requests_html import HTML
#from requests_html import HTMLSession
from bs4 import BeautifulSoup
import re

## Lấy danh sách các link 

In [ ]:
arr_data = []
for page in range(1, 2):
  link ="https://www.goodreads.com/list/show/7.Best_Books_of_the_21st_Century?page="+ str(page)
  page = requests.get(link)
  soup = BeautifulSoup(page.content, 'html.parser')
  arr_link = soup.find_all("a",class_='bookTitle')
  for i in range(len(arr_link)):
    link ="https://www.goodreads.com" + arr_link[i]['href']
    arr_data.append(link)

In [ ]:
print("Tổng số link: " + str(len(arr_data)))
df_link=pd.DataFrame(arr_data)
links = df_link[0].values

Tổng số link: 100


## Xử lý từng link sách

In [ ]:
arr_data = []
for i in range(10):
  link = links[i]
  page = requests.get(link)
  soup = BeautifulSoup(page.content, 'html.parser')
  if soup:
    id = link.split('show/')[-1].split('-')[0].split('.')[0]
    title = soup.find(id='bookTitle')
    serie  = float('nan')
    author  = float('nan')
    author_link  = float('nan')
    rating_count  = float('nan')
    review_count  = float('nan')
    number_of_pages  = float('nan')
    date_published  = float('nan')
    publisher  = float('nan')
    original_title  = float('nan')
    genre_and_votes  = float('nan')
    isbn  = float('nan')
    isbn13  = float('nan')
    setting  = float('nan')
    characters  = float('nan')
    description = float('nan')
    awards = float('nan')
    rating = float('nan')
    rating = soup.find(itemprop = 'ratingValue').text.strip()
    serie = soup.find(id='bookSeries')#
    if serie:
      serie = serie.text.strip()
    author_section = soup.find(id='bookAuthors')
    if author_section:
      author_section = author_section.find(class_='authorName')
    if author_section:
      author = author_section.text.strip()
      author_link = author_section['href'] #
    title = soup.find(id='bookTitle')
    if title:
      title = title.text.strip()
    review_section = soup.find(id='reviewControls')
    if review_section:
      review_section = review_section.find(class_='reviewControls--left greyText')
    if review_section:
      temp = re.findall(r'[\w,]+', review_section.text)
      rating_count = float(temp[0].replace(',', '')) #
      review_count = float(temp[2].replace(',', '')) #
    if soup.find(itemprop='numberOfPages'):
      number_of_pages = int(soup.find(itemprop='numberOfPages').text.split(' ')[0].strip()) #
    temp = soup.find(id='details')
    if temp:
      #print(temp)
      temp = temp.find_all(class_='row')
      if len(temp) > 1:
        temp = temp[1]
        date_published = temp.text.strip().split('\n')[1].strip()
        if len(temp.text.strip().split('\n')) > 2:
          #print(temp.text.strip().split('\n'))
          publisher = temp.text.strip().split('\n')[2].split('by')
          if len(publisher) >= 2:
            publisher = publisher[1].strip()
    a = soup.find(id='bookDataBox')
    if a:
      a = a.find_all(class_='infoBoxRowTitle')
    b = soup.find(id='bookDataBox')
    if b:
      b = b.find_all(class_='infoBoxRowItem')
    if a and b:
      for i in range(len(a)):
        if a[i].text.strip() == 'Original Title':
          original_title = b[i].text.strip()
        if a[i].text.strip() == 'ISBN':
          isbn = b[i].text.strip()
        if a[i].text.strip() == 'Characters':
          characters = b[i].text.strip()
        if a[i].text.strip() == 'Setting':
          setting = b[i].text.strip()
        if a[i].text.strip() == 'Literary Awards':
          awards = b[i].text.strip()
      if isbn == isbn:
        temp = re.findall(r'[\w]+', isbn)
        if (len(temp) == 3):
          isbn = temp[0]
          isbn13 = temp[2]
    genre_and_votes_arr = []
    temp = soup.find_all(class_='elementList ')
    for i in temp:
      left = i.find(class_='left').find_all(class_='actionLinkLite bookPageGenreLink')
      left_arr = []
      for j in left:
        left_arr.append(j.text.strip())
        left_text = '-'.join(left_arr)
        right_text = i.find(class_='right').text.split(' ')[12].strip().replace(',', '')
      genre_and_votes_arr.append(str(left_text) + ' ' + str(right_text))
    genre_and_votes = ', '.join(genre_and_votes_arr)
    description = soup.find(id='description')
    if description:
      description = description.text.strip()
    combine_data = [id,title,link, serie, author, author_link, rating_count,
                    review_count, number_of_pages, date_published, publisher, original_title,
                    genre_and_votes, isbn, isbn13, setting, characters, description,awards,rating]
    arr_data.append(combine_data.copy())

In [ ]:
feature = ['id','title','link', 'serie', 'author', 'author_link', 'rating_count',
                    'eview_count', 'number_of_pages', 'date_published', 'publisher', 'original_title',
                    'genre_and_votes', 'isbn', 'isbn13', 'setting', 'characters', 'description','awards','rating']
df= pd.DataFrame(arr_data,columns=feature)

In [ ]:
df['genre_and_votes'][6]

'Fiction 12877, Historical-Historical Fiction 11633, Historical 2275, Adult 1381, Adult Fiction 1242, Contemporary 1005, Classics 920, Audiobook 902, Book Club 826'

In [ ]:
df.to_csv('Data.csv')